### Importando Bibliotecas e separando conjuntos de treino e validação

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

### Criando conjuntos de teste e validação

In [ ]:
OBS: REALIZAR O TREINAMENTO COM AS FOTOS EM CORES DE CINZA PODE TRAZER MAIOR ACURÁCIA COM CORES DE FILMAENTOS DIFERNETES?

In [48]:
# Caminho para o diretório que contém as subpastas de classes
diretorio = 'imagens'

# Inicializa o gerador de dados com a opção de rescale e divisão de validação
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # reserva 20% das imagens para validação
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    
)

# Cria o gerador de treino
train_generator = datagen.flow_from_directory(
    diretorio,
    target_size=(224, 224),  # redimensiona as imagens para 224x224
    batch_size=20,
    class_mode='binary',
    subset='training',  # define este conjunto como sendo de treinamento,
    shuffle=True
)

# Cria o gerador de validação
validation_generator = datagen.flow_from_directory(
    diretorio,
    target_size=(224, 224),  # redimensiona as imagens para 224x224
    batch_size=20,
    class_mode='binary',
    subset='validation',  # define este conjunto como sendo de validação
    shuffle=True
)

Found 1120 images belonging to 2 classes.
Found 279 images belonging to 2 classes.


### Verificando algumas imagens

In [ ]:
import matplotlib.pyplot as plt

# Gera e exibe imagens de augmentação
def display_augmented_images(generator, num_images=5):
    fig, ax = plt.subplots(nrows=1, ncols=num_images, figsize=(15, 15))
    for i in range(num_images):
        # Obtem um batch de imagens
        img, label = next(generator)
        # Exibe a primeira imagem do batch (pois o tamanho do batch é 1)
        ax[i].imshow(img[0])
        ax[i].axis('off')
    plt.show()

In [ ]:
display_augmented_images(train_generator, num_images=10)

In [51]:
classes_train = train_generator.class_indices
classes_validation = validation_generator.class_indices

print("Classes no conjunto de treinamento:", classes_train)
print("Classes no conjunto de validação:", classes_validation)

Classes no conjunto de treinamento: {'corretas': 0, 'erradas': 1}
Classes no conjunto de validação: {'corretas': 0, 'erradas': 1}


In [52]:
# Contagem de imagens por classe no conjunto de treinamento
class_indices_train = train_generator.class_indices
labels_train = train_generator.labels
count_images_per_class_train = {class_name: labels_train.tolist().count(class_idx) for class_name, class_idx in class_indices_train.items()}

print("Quantidade de fotos por classe no conjunto de treinamento:", count_images_per_class_train)

# Contagem de imagens por classe no conjunto de validação
class_indices_validation = validation_generator.class_indices
labels_validation = validation_generator.labels
count_images_per_class_validation = {class_name: labels_validation.tolist().count(class_idx) for class_name, class_idx in class_indices_validation.items()}

print("Quantidade de fotos por classe no conjunto de validação:", count_images_per_class_validation)


Quantidade de fotos por classe no conjunto de treinamento: {'corretas': 1028, 'erradas': 92}
Quantidade de fotos por classe no conjunto de validação: {'corretas': 256, 'erradas': 23}


In [53]:
# Carregar a VGG16 como base do modelo, sem incluir as camadas superiores
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar as camadas do modelo base para não serem treinadas
for layer in base_model.layers:
    layer.trainable = False

# Adicionar novas camadas no topo para a tarefa específica
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Use dropout para reduzir o overfitting

predictions = layers.Dense(1, activation='sigmoid')(x)  # Use sigmoid para classificação binária

# Compilar o modelo
model = Model(inputs=base_model.input, outputs=predictions)
# model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,560,769 (105.14 MB)

 Trainable params: 12,846,081 (49.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.000001), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=10,  # Depende do tamanho do seu conjunto de dados
    epochs=10,            # Ajuste conforme necessário
    validation_data=validation_generator,
    validation_steps=20,  # Depende do tamanho do seu conjunto de validação
    verbose=2)

Epoch 1/10
40/40 - 58s - 1s/step - accuracy: 0.7837 - loss: 0.4811 - val_accuracy: 0.9176 - val_loss: 0.2773
Epoch 2/10


2024-04-04 17:57:01.950500: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-04 17:57:17.973611: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


40/40 - 30s - 745ms/step - accuracy: 0.9031 - loss: 0.3176 - val_accuracy: 0.9176 - val_loss: 0.2514
Epoch 3/10


2024-04-04 17:57:31.776223: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


40/40 - 57s - 1s/step - accuracy: 0.9175 - loss: 0.2891 - val_accuracy: 0.9176 - val_loss: 0.2446
Epoch 4/10


...

2024-04-04 17:58:58.630516: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


40/40 - 57s - 1s/step - accuracy: 0.9187 - loss: 0.2689 - val_accuracy: 0.9176 - val_loss: 0.2415
Epoch 6/10


2024-04-04 17:59:55.405090: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-04 18:00:11.635258: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


40/40 - 30s - 752ms/step - accuracy: 0.9187 - loss: 0.2361 - val_accuracy: 0.9176 - val_loss: 0.2313
Epoch 7/10


2024-04-04 18:00:25.505200: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
